In [3]:
import mido
import time

In [4]:
with mido.open_output("to Max 1") as midi_out:
    while True:
        msg = mido.MetaMessage("text", text=f"it is {time.time()}")
        midi_out.send(msg)
        time.sleep(1)

ValueError: 'message' longer than 3 bytes but does not start with 0xF0.